<a href="https://colab.research.google.com/github/WeimusT/Quantitative_trading/blob/main/Statistical_Factor_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

The Capital Asset Pricing Model (CAPM) is a model to estimate the return of a stock ($E[R]$) using market return ($R_m$), risk free return ($R_f$) and the volatility of the stock ($\beta$). It is defined as 

$$E[R]=R_f + \beta(R_m-R_f)$$.



Arbitrage Pricing T

Factor model

[Fama French Model](https://www.investopedia.com/terms/f/famaandfrenchthreefactormodel.asp)

In [ ]:
# Calculate last 30 day daily beta of Tenable

In [ ]:
import math
import numpy as np
from numpy.linalg import eig, eigh
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import Ridge
import time

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
lookback = 252
numFactors = 5
topN = 50

In [ ]:
# Load S&P 600 small cap stocks
df = pd.read_table("/content/drive/MyDrive/data/IJR_20080114.txt")
df["Date"] = df["Date"].astype("int")
df.set_index("Date", inplace=True)
df.fillna(method="ffill", inplace=True)
df.sort_index(inplace=True)
df

,ABCW,ABFS,ABM,ACAT,ACLS,ACO,ACTL,ADPT,AEIS,AGL,...,WTS,WW,WWW,WXS,XRIT,ZEP,ZLC,ZNT,ZQK,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
20040115,22.1609,30.9237,16.4747,26.4522,11.32,23.3373,26.93,10.10,27.430,21.6689,...,22.6328,24.3135,13.8953,NaN,11.8467,NaN,26.115,19.7760,8.865,NaN
20040116,22.2135,30.2284,16.4129,25.7508,12.56,19.5716,27.92,10.50,27.390,21.5655,...,22.2980,23.9795,13.7789,NaN,11.9759,NaN,26.015,19.7346,8.970,NaN
20040120,22.6166,30.6230,16.5188,26.3465,13.22,22.5940,28.00,11.05,27.500,21.1893,...,22.4851,24.2447,13.6109,NaN,12.6120,NaN,26.125,20.9153,8.965,NaN
20040121,22.7393,30.9425,16.4394,26.4128,12.77,20.7356,27.00,10.68,26.780,21.1046,...,22.5540,24.0188,13.6368,NaN,12.7014,NaN,26.200,20.8976,8.935,NaN
20040122,22.6867,31.4311,16.2541,25.6153,12.23,20.0216,27.05,10.18,25.578,20.9259,...,22.6033,24.1858,13.4623,NaN,12.6815,NaN,26.760,20.7441,8.775,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080108,21.1800,18.2800,19.0500,8.1200,4.25,32.4300,12.20,3.13,10.550,18.0000,...,27.0800,43.1000,21.3100,34.00,11.2400,13.55,15.270,42.5800,7.491,16.3100
20080109,21.7200,18.9200,19.1101,7.6800,4.28,33.3600,12.52,3.29,10.630,18.0700,...,27.1000,43.3100,22.0300,32.96,11.3484,13.54,14.780,43.4700,7.820,17.1576
20080110,22.2800,18.9600,19.3400,8.2800,4.38,33.0400,12.44,3.25,11.230,18.1400,...,27.9200,44.3200,21.8100,33.12,11.4500,14.54,14.350,43.6200,8.020,17.5600


In [ ]:
dailyret = df.pct_change()
positionsTable = np.zeros(df.shape)
end_index = df.shape[0]
dailyret

,ABCW,ABFS,ABM,ACAT,ACLS,ACO,ACTL,ADPT,AEIS,AGL,...,WTS,WW,WWW,WXS,XRIT,ZEP,ZLC,ZNT,ZQK,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
20040115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20040116,0.002374,-0.022484,-0.003751,-0.026516,0.109541,-0.161360,0.036762,0.039604,-0.001458,-0.004772,...,-0.014793,-0.013737,-0.008377,NaN,0.010906,NaN,-0.003829,-0.002093,0.011844,NaN
20040120,0.018147,0.013054,0.006452,0.023133,0.052548,0.154428,0.002865,0.052381,0.004016,-0.017445,...,0.008391,0.011059,-0.012193,NaN,0.053115,NaN,0.004228,0.059829,-0.000557,NaN
20040121,0.005425,0.010433,-0.004807,0.002516,-0.034039,-0.082252,-0.035714,-0.033484,-0.026182,-0.003997,...,0.003064,-0.009318,0.001903,NaN,0.007088,NaN,0.002871,-0.000846,-0.003346,NaN
20040122,-0.002313,0.015791,-0.011272,-0.030194,-0.042287,-0.034434,0.001852,-0.046816,-0.044884,-0.008467,...,0.002186,0.006953,-0.012796,NaN,-0.001567,NaN,0.021374,-0.007345,-0.017907,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080108,-0.049372,-0.054320,-0.022074,-0.096774,-0.044944,-0.038256,-0.030977,-0.027950,-0.037409,-0.013699,...,-0.035956,-0.024666,-0.027385,-0.018759,-0.005310,-0.014545,0.015968,-0.048279,-0.030918,-0.035864
20080109,0.025496,0.035011,0.003155,-0.054187,0.007059,0.028677,0.026230,0.051118,0.007583,0.003889,...,0.000739,0.004872,0.033787,-0.030588,0.009644,-0.000738,-0.032089,0.020902,0.043919,0.051968
20080110,0.025783,0.002114,0.012030,0.078125,0.023364,-0.009592,-0.006390,-0.012158,0.056444,0.003874,...,0.030258,0.023320,-0.009986,0.004854,0.008953,0.073855,-0.029093,0.003451,0.025575,0.023453


In [ ]:
for t in tqdm(np.arange(lookback+1, end_index)):
  # For each day in the testing period. Build a model base the lookback period.
  R = dailyret.iloc[t-lookback+1:t,].T # Get the lookback period return for each day in the test period
  hasData = np.where(R.notna().all(axis=1))[0]
  R.dropna(inplace=True)
  pca = PCA()
  X = pca.fit_transform(R.T)[:, :numFactors]
  X = sm.add_constant(X)
  y1 = R.T
  clf = MultiOutputRegressor(LinearRegression(fit_intercept=False), n_jobs=4).fit(X, y1)
  Rexp = np.sum(clf.predict(X), axis=0)
  R = dailyret.iloc[t-lookback+1:t+1,].T
  idxSort = Rexp.argsort()
  positionsTable[t, hasData[idxSort[np.arange(0, topN)]]]=-1
  positionsTable[t, hasData[idxSort[np.arange(-topN, -1)]]] = 1

100%|██████████| 753/753 [14:17<00:00,  1.14s/it]


In [ ]:
capital=np.nansum(np.array(abs(pd.DataFrame(positionsTable)).shift()), axis=1)
positionsTable[captial==0, ]=0
capital[capital==0]=1
ret = np.nansum(np.array(pd.DataFrame(positionsTable).shift())*np.array(dailyret), axis=1)/capital
avgret = np.nanmean(ret)*252
avgstdev = np.nanstd(ret)*math.sqrt(252)
Sharpe = avgret / avgstdev
print(avgret)
print(avgstdev)
print(Sharpe)